In [ ]:
# | default_exp server

In [ ]:
# | export

import importlib
import sys
import asyncio
from typing import *
from contextlib import contextmanager
from pathlib import Path
import threading
import signal
from contextlib import ExitStack

import multiprocessing
from fastcore.meta import delegates
from fastcore.basics import patch

from fastkafka.application import FastKafka
from fastkafka.testing import change_dir

from fastkafka._components.logger import get_logger

In [ ]:
import os
from time import sleep
from contextlib import contextmanager
from pydantic import BaseModel
from tempfile import TemporaryDirectory
from fastkafka._components.logger import supress_timestamps

In [ ]:
# | notest

# allows async calls in notebooks

import nest_asyncio

In [ ]:
# | notest

nest_asyncio.apply()

In [ ]:
# | export

logger = get_logger(__name__)

In [ ]:
supress_timestamps()
logger = get_logger(__name__, level=20)
logger.info("ok")

[INFO] __main__: ok


In [ ]:
def create_test_app():
    app = FastKafka(bootstrap_servers="tvrtko-fastkafka-kafka-1:9092")

    class MyMessage(BaseModel):
        msg: str

    @app.consumes()
    def on_my_topic(msg: MyMessage):
        pass
    
    return app

In [ ]:
# | export

class ServerProcess():
    
    def __init__(self, app: FastKafka):
        if app._is_started:
            raise RuntimeError(f"FastKafka app was already started!")
            
        self.app = app
        self.should_exit = False
    
    
    def run(self) -> None:
        return asyncio.run(self._serve())

    async def _serve(self) -> None:
        self._install_signal_handlers()

        async with self.app:
            await self._main_loop()

    def _install_signal_handlers(self) -> None:
        if threading.current_thread() is not threading.main_thread():
            raise RuntimeError()

        loop = asyncio.get_event_loop()

        HANDLED_SIGNALS = (
            signal.SIGINT,  # Unix signal 2. Sent by Ctrl+C.
            signal.SIGTERM,  # Unix signal 15. Sent by `kill <pid>`.
        )
        
        def handle_exit(sig: int) -> None:
            self.should_exit = True
        
        for sig in HANDLED_SIGNALS:
            loop.add_signal_handler(sig, handle_exit, sig)
            
            
    async def _main_loop(self) -> None:
        while not self.should_exit:
            await asyncio.sleep(0.1)


    @contextmanager
    def run_in_process(self) -> Generator[None, None, None]:

        def create_and_run(app=self.app):
            server = ServerProcess(app=app)
            server.run()

        with run_in_process_until_terminate(create_and_run):
            yield
    
@contextmanager
def run_in_process_until_terminate(target: Callable[..., Any]) -> Generator[None, None, None]:
    p = multiprocessing.Process(target=target)
    try:
        p.start()
        yield
    except Exception as e:
        logger.warning(f"Exception raised {e=}")
        
    finally:
        for i in range(6):
            p.terminate()
            p.join(5)
            if p.exitcode is not None:
                break
            else:
                logger.warning("Process not terminated, retrying...")
        if p.exitcode is None:
            logger.warning("Killing the process...")
            p.kill()
            p.join()
            logger.warning("Process killed!")
                
        p.close()


In [ ]:
for _ in range(5):
    print("*"*100)
    app = create_test_app()
    server = ServerProcess(app=app)
    with server.run_in_process():
        print(1)
        sleep(1)
        with server.run_in_process():
            print(2)
            sleep(1)
            with server.run_in_process():
                print(3)
                sleep(1)
                print(4)
            sleep(1)
            print(5)
        sleep(1)
        print(6)

****************************************************************************************************
[INFO] fastkafka._components.asyncapi: Keeping the old async specifications at: '/work/fastkafka/nbs/asyncapi/spec/asyncapi.yml'
[INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop() starting...
[INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Consumer created using the following parameters: {'bootstrap_servers': 'tvrtko-fastkafka-kafka-1:9092', 'auto_offset_reset': 'earliest', 'max_poll_records': 100}
[INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Consumer started.
[INFO] aiokafka.consumer.subscription_state: Updating subscribed topics to: frozenset({'my_topic'})
[INFO] aiokafka.consumer.consumer: Subscribed to topic(s): {'my_topic'}
[INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Consumer subscribed.
[INFO] aiokafka.consumer.group_coordinator: Metadata for topic has cha

[INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Consumer started.
[INFO] aiokafka.consumer.subscription_state: Updating subscribed topics to: frozenset({'my_topic'})
[INFO] aiokafka.consumer.consumer: Subscribed to topic(s): {'my_topic'}
[INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Consumer subscribed.
[INFO] aiokafka.consumer.group_coordinator: Metadata for topic has changed from {} to {'my_topic': 1}. 
2
[INFO] fastkafka._components.asyncapi: Keeping the old async specifications at: '/work/fastkafka/nbs/asyncapi/spec/asyncapi.yml'
[INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop() starting...
[INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Consumer created using the following parameters: {'bootstrap_servers': 'tvrtko-fastkafka-kafka-1:9092', 'auto_offset_reset': 'earliest', 'max_poll_records': 100}
[INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_con

[INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Consumer subscribed.
[INFO] aiokafka.consumer.group_coordinator: Metadata for topic has changed from {} to {'my_topic': 1}. 
3
[INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Consumer stopped.
[INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop() finished.
4
[INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Consumer stopped.
[INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop() finished.
5
[INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Consumer stopped.
[INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop() finished.
6


In [ ]:
# | export

class Server(ServerProcess):
    
    def __init__(self, app: FastKafka, num_workers: Optional[int]=None):
        ServerProcess.__init__(self, app=app)
        self.num_workers: int = num_workers if isinstance(num_workers, int) else multiprocessing.cpu_count()

    async def _serve(self) -> None:
        self._install_signal_handlers()
        with ExitStack() as stack:
            server_processes = [ServerProcess(app=self.app) for _ in range(self.num_workers)]
            for server_process in server_processes:                
                stack.enter_context(server_process.run_in_process())
            await self._main_loop()
            
    @contextmanager
    def run_in_process(self) -> Generator[None, None, None]:

        def create_and_run(app=self.app, num_workers=self.num_workers):
            server = Server(app=app, num_workers=num_workers)
            server.run()

        with run_in_process_until_terminate(create_and_run):
            yield

In [ ]:
with TemporaryDirectory() as d:
    with change_dir(d):
        server = Server(app)
        with server.run_in_process():
            print("I'm in!")
            sleep(5)
            print("Going out...")

print("I'm out!")

[INFO] fastkafka._components.asyncapi: Old async specifications at '/tmp/tmp0fjohcht/asyncapi/spec/asyncapi.yml' does not exist.
[INFO] fastkafka._components.asyncapi: New async specifications generated at: '/tmp/tmp0fjohcht/asyncapi/spec/asyncapi.yml'
[INFO] fastkafka._components.asyncapi: Keeping the old async specifications at: '/tmp/tmp0fjohcht/asyncapi/spec/asyncapi.yml'
[INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop() starting...
[INFO] fastkafka._components.asyncapi: Keeping the old async specifications at: '/tmp/tmp0fjohcht/asyncapi/spec/asyncapi.yml'
[INFO] fastkafka._components.asyncapi: Keeping the old async specifications at: '/tmp/tmp0fjohcht/asyncapi/spec/asyncapi.yml'
I'm in!
[INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop() starting...
[INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Consumer created using the following parameters: {'bootstrap_servers': 'tvrtko-fastkafka-kafka-1:9092'